In [1]:
import pandas as pd
import talib
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
btc_df = pd.read_csv('BTCUSDC-1m-2years.csv') 
btc_df['timestamp'] = pd.to_datetime(btc_df['timestamp'],dayfirst=True )
btc_df.set_index('timestamp', inplace=True)

In [ ]:
import numpy as np
from sklearn.linear_model import RidgeClassifier

class RandomizedSignatureDrift:
    def __init__(self, rd=50, rm=0.05, rv=0.03, alpha=1e-3, tw=120):
        """
        Initialize parameters based on Table 1 and Section 2.2.
        
        Args:
            rd (int): Reservoir dimension (r_d) [cite: 831]
            rm (float): Mean of random projection entries (r_m) [cite: 188]
            rv (float): Variance of random projection entries (r_v) [cite: 188]
            alpha (float): Regularization for Ridge regression [cite: 225]
            tw (int): Rolling window size / start index (t_w) [cite: 200]
        """
        self.rd = rd
        self.rm = rm
        self.rv = rv
        self.alpha = alpha
        self.tw = tw
        self.activation = np.tanh # Common choice for sigma in Reservoir Computing

    def _generate_reservoir_system(self, input_dim):
        """
        Generates the random matrices A_i and biases b_i for Eq (4)[cite: 183].
        
        Input dimension is d+1 (stocks + time).
        There is one matrix A and bias b for each input dimension component.
        """
        # A_i: Shape (input_dim, rd, rd)
        # Entries follow N(rm, rv) [cite: 188]
        # We use standard deviation = sqrt(rv) for numpy's normal function
        self.A = np.random.normal(loc=self.rm, scale=np.sqrt(self.rv), 
                                  size=(input_dim, self.rd, self.rd))
        
        # b_i: Shape (input_dim, rd)
        # Entries follow N(0, 1) [cite: 188]
        self.b = np.random.normal(loc=0, scale=1, 
                                  size=(input_dim, self.rd))

    def compute_signatures(self, X_augmented):
        """
        Computes the randomized signature (Reservoir state Y) via Euler scheme.
        Eq (4): dR_t = sum(sigma(A_i R_t + b_i) dX^i_t) [cite: 183]
        """
        T, input_dim = X_augmented.shape
        
        # Initialize R_0 ~ N(0, I) [cite: 184]
        R = np.zeros((T, self.rd))
        R[0] = np.random.normal(0, 1, self.rd)
        
        # Calculate increments dX (simple difference for discrete time)
        # Note: dX[t] drives the change from R[t] to R[t+1]
        dX = np.diff(X_augmented, axis=0, prepend=0) 

        # Euler Discretization Loop
        for t in range(T - 1):
            update_sum = np.zeros(self.rd)
            
            # Summation over input dimensions i=0 to d
            for i in range(input_dim):
                # Linear projection: A_i * R_t + b_i
                proj = np.dot(self.A[i], R[t]) + self.b[i]
                # Activation and multiplication by increment dX^i
                update_sum += self.activation(proj) * dX[t, i]
            
            R[t+1] = R[t] + update_sum
            
        return R

    def predict(self, prices):
        """
        Main algorithm from Appendix 3[cite: 889].
        """
        # 1. Preprocessing: Log returns (Leading increments) [cite: 172]
        # Normalizing by initial value is mentioned in [cite: 172]
        prices = np.array(prices)
        log_prices = np.log(prices / prices[0]) 
        X_log_returns = np.diff(log_prices, axis=0) # Length T-1
        y_returns = np.where(X_log_returns>0, 1,-1)
        
        # 2. Data Augmentation: Add time as 0-th dimension [cite: 195]
        T_len, _ = X_log_returns.shape
        time_index = np.arange(T_len).reshape(-1, 1) #* (1.0/T_len) # Normalized time
        X_augmented = np.hstack([time_index, X_log_returns])
        
        # 3. Initialize Reservoir System (Random Matrices)
        self._generate_reservoir_system(input_dim=X_augmented.shape[1])
        
        # 4. Compute Reservoir States (Y) [cite: 889]
        Y = self.compute_signatures(X_augmented)
        
        # 5. Set Split Time (t_s) [cite: 229, 889]
        # Usually 10% of data, ensuring t_s >> t_w
        t_s = int(T_len * 9 / 10)
        if t_s <= self.tw:
            t_s = self.tw + 10
            
        predictions = []
        
        # 6. Main Loop: Train and Predict [cite: 889]
        # We iterate to predict the NEXT return. 
        print(f"Starting training loop from t={t_s} to {T_len-1}...")
        # Define expanding window training sets
        # Inputs: Reservoir states from t_w to t-1

        X_train = Y[self.tw : t_s-120]
        
        # Targets: Log returns from t_w+1 to t (Predicting 1 step ahead)
        # We want to map Y[k] -> Return[k+1]
        y_train = y_returns[self.tw + 1 : t_s-120 + 1]
        
        # Ridge classifier Readout [cite: 225]
        ridge = RidgeClassifier(alpha=self.alpha, fit_intercept=True)
        ridge.fit(X_train, y_train)
        for t in range(t_s, T_len-1):
            
            
            # Predict next step: \hat{X}_{t+1} = M_t(Y_t) [cite: 889]
            # We use the current reservoir state Y[t] to predict return at t+1
            current_reservoir = Y[t].reshape(1, -1)
            pred = ridge.predict(current_reservoir)
            predictions.append(pred.flatten())
            
        y_true = y_returns[t_s+1 : T_len]
        ascore = accuracy_score(y_true, predictions)
        return ascore


In [42]:
prices = btc_df.close.values
model = RandomizedSignatureDrift(rd=50, rm=0.05, rv=0.03, tw=120)
predicted_returns = model.predict(prices[:,np.newaxis])
print(predicted_returns)

Starting training loop from t=828143 to 920158...


/opt/anaconda3/envs/mlkaggle/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5180946791862285
